# 2.3 优化提示词改善答疑机器人回答质量

## 🚄 前言
想象一下，你正在给同事指派一个工作任务，如果只给了他一句话来描述这个需求，他的任务完成效果可能很难达到你的预期。但如果你提供了明确的目标、建议的思考方向和执行策略等更多参考信息，他更有可能以高标准完成任务。使用大语言模型也一样，你的提问方式（提示词）决定了大语言模型的回答质量。

在前一章节中，你已经开发了一个关于公司规章制度的答疑机器人。但是你的用户可能对机器人有更多的要求，比如希望语气、回复格式上有所调整。这些要求，你都可以通过改进提示词来完成。

## 🍁 课程目标

学完本课程后，你将能够：

- 学习通过构造更高质量的提示词来获得更好的大模型回复
- 学习在工程中应用大模型处理多种不同类型的任务

## 📖 课程目录
本章节将引导你通过改进提示词、将多个任务编排到一起等手段，逐步提升机器人的回答质量：

- 1.&nbsp;前文回顾
- 2.&nbsp;通过优化提示词来调整大模型的回答
- 3.&nbsp;如何改进提示词
    - 3.1 使用提示词框架
    - 3.2 持续学习提示词技巧
- 4.&nbsp;为答疑机器人扩展更多技能使他成为一位多面手
    - 4.1 通过简单提示词进行问题初步分类
    - 4.2 优化分类提示词
    - 4.3 实现路由模块并应用到答疑机器人中
- 5.&nbsp;拓展阅读：利用百炼工作流来完成文档自动审阅和修改
    - 5.1 创建一个工作流应用
    - 5.2 按照如下图所示搭建工作流并【发布】
    - 5.3 在我的应用界面点击【调用】按钮找到调用代码
    - 5.4 调用代码封装


## 💻 1. 前文回顾

在上一章节，你已经完成了一个基本可用的答疑机器人，为了方便调用，你将其封装成了几个函数，并保存在了 `chatbot/rag.py` 中。
现在你可以通过如下代码来快速调用。

In [6]:
# 加载百炼的 API Key 用于调用通义千问大模型
import os
from config.load_key import load_key
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


In [7]:
from chatbot import rag
# 上一章节已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index()
query_engine = rag.create_query_engine(index=index)

rag.ask('我们公司项目管理应该用什么工具', query_engine=query_engine)

公司项目管理可以使用如Jira或Trello这样的项目管理软件。这些工具能够帮助团队更好地规划和跟踪项目的进度，确保任务按时完成。同时，它们还支持任务分配、时间跟踪和团队协作等功能，有助于提高工作效率。

在答疑机器人上线后，很多同事向你提出了各种改进需求，比如：

- *希望在问答疑机器人我们公司的写作、项目管理工具时，能把对应的下载地址也给出来。*
- ...

## 2. 优化提示词以改善大模型回答质量

虽然你可以让同事在提问时，指明要求下载地址，但是并不是所有的同事都会记得这一注意事项。你希望在类似的问题上，都可以自动给出下载地址。

因此，你可以考虑改进下程序，在收到用户问题后，自动补充一些关于回答内容的要求：

In [8]:
from llama_index.llms.dashscope import DashScope

def ask_v2(question, query_engine):
    streaming_response = query_engine.query(question + "如果是工具咨询类问题，请务必给出下载地址链接。")
    streaming_response.print_response_stream()

ask_v2('我们公司项目管理应该用什么工具', query_engine=query_engine)

对于项目管理，推荐使用Jira或Trello。这两个工具都能帮助团队有效地管理项目进度，确保任务按时完成。Jira更适合复杂项目管理和软件开发团队，而Trello则以其简洁直观的界面受到广泛欢迎，适合各种规模的项目。

- Jira: 可以访问 [Atlassian官网](https://www.atlassian.com/software/jira) 下载。
- Trello: 可以访问 [Trello官网](https://trello.com/) 下载。

在上面代码调整中引入的 `question + "如果是工具查询类问题，请务必给出下载地址链接。"`可以理解成一个提示词模板。

借助 LlamaIndex 中的 rag 应用背后，其实已经有一个提示词模板，你可以去调整该模板，来获得更好的效果。

In [9]:
# 查看原始模板
query_engine = index.as_query_engine(
  streaming=True,
  llm=DashScope(model_name="qwen-plus")
)
print(f"LlamaIndex原始prompt模板为：\n{query_engine.get_prompts()['response_synthesizer:text_qa_template'].default_template.template}")

LlamaIndex原始prompt模板为：
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


In [10]:
from llama_index.core import PromptTemplate

prompt_template_string = (
    "你是公司小蜜，你需要简明扼要的回答大家的问题"
    "注意事项：\n"
    "1. 根据上下文信息而非先验知识来回答问题。\n"
    "2. 如果是工具咨询类问题，请务必给出下载地址链接。\n"
    "以下是参考信息。"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "问题：{query_str}\n。"
    "回答："
)
prompt_template = PromptTemplate(prompt_template_string)

query_engine = index.as_query_engine(streaming=True, llm=DashScope(model_name="qwen-plus"))
query_engine.update_prompts({"response_synthesizer:text_qa_template": prompt_template})

def ask_v3(question, query_engine):
    streaming_response = query_engine.query(question + "如果是工具查询类问题，请务必给出下载地址链接。")
    streaming_response.print_response_stream()

ask_v3('我们公司项目管理应该用什么工具', query_engine=query_engine)

公司项目管理推荐使用的工具包括Jira和Trello。您可以根据团队的具体需求选择合适的工具。

- **Jira**：适合复杂项目管理和敏捷开发团队。[下载地址](https://www.atlassian.com/software/jira/downloads)
- **Trello**：适合简单项目管理和任务跟踪。[下载地址](https://trello.com/)

请根据您的项目特点选择合适的工具，并确保团队成员都能熟练使用所选工具，以提高项目管理效率。

在上面的代码中，我们调整了应用的默认提示词模板，使其更符合同事的需求：

- 在最开头指明了机器人的人设，并且要求其回复风格要简明扼要；
- 增加注意事项，针对工具咨询类问题，会给出具体下载链接。


## 3. 如何写好提示词

### 3.1. 使用提示词框架
在前面的实验中你会发现，和人与人之间的交流一样，你给到大模型的任务描述越清晰具体，大模型的回复越符合你的需求。因此你可以考虑在提示词中补充更多信息，比如：目标、背景、受众、输出格式等等。参考一些提示词框架，能帮助你构建一个完整、有效的提示词。

提示词框架示例：

|要素|含义|
|----|----|
|任务目标（Object）|明确要求大模型完成什么任务，让大模型专注具体目标|
|上下文（Context）|任务的背景信息，比如操作流水，任务场景等等，确保大模型理解讨论的范围|
|角色（Role）|大模型扮演的角色，或者强调大模型应该使用的语气，写作风格等等，明确大模型回应的预期情感|
|受众（Audience）|明确大模型针对的特定受众，约束大模型的应答风格|
|样例（Sample）|让大模型参考的具体案例，大模型会从中抽象出实现方案、需要注意的具体格式等等信息，尽可能贴合开发者的需要|
|输入数据（Input Parameters）|一般用明确的引导词来说明外部输入的数据，如果有多个具体含义的参数，也需要明确向大模型说明参数的作用。|
|输出格式（Output Format）|明确指定输出的格式，输出类型，枚举值的范围。通常也会明确指出不需要输出的内容，不期望的信息。可以结合样例来进一步明确输出的格式和输出方法。|


事实上，提示词框架远不止这一种，许多问题分析的思维范式都可以用来帮助我们更清晰具体的描述需求。例如，SWOT分析法、5W2H分析法等，这些方法都能引导大模型从不同角度思考问题，从而更全面地提供信息。

你也可以考虑使用阿里云百炼提供的[提示词自动优化工具](https://bailian.console.aliyun.com/#/prompt-manage)，来帮助你完善提示词。

<img src="https://img.alicdn.com/imgextra/i3/O1CN014JCCqn22zX6xB4tt3_!!6000000007191-0-tps-2068-1052.jpg" width="700"/>

### 3.2. 持续学习提示词技巧

构建有效的提示词是构建一个好的大模型应用的关键。因此持续学习、和尝试不同的提示词技巧是非常有必要的。

这里是一个开源的[提示词工程指南](https://www.promptingguide.ai/zh)，非常建议你持续学习和尝试。该指南包含了目前很多实用的提示词技巧，如：
- [少量样本提示((Few-Shot Prompting)](https://www.promptingguide.ai/techniques/fewshot)
- [链式思考（Chain-of-Thought Prompting）](https://www.promptingguide.ai/zh/techniques/cot)
- ...

## 🛠️ 4. 解锁大模型的更多技能

随着越来越多的同事使用你开发的答疑机器人，他们开始发现答疑机器人不只是可以用来查资料，还可以用来帮助他们审查文档、翻译文档。

比如你可以让机器人帮你检查一段文档是否存在问题：

In [11]:
rag.ask('请帮我检查下这段文档：地球和月亮之间存在着玩有引力，所以月亮会绕着地球。', query_engine=query_engine)

这段文档中有一个小错误，正确的表述应该是：“地球和月亮之间存在着万有引力，所以月亮会绕着地球转。” 

修改建议：
- 将“玩有引力”改为“万有引力”。
- 将句尾的“。”改为“转。”以使句子更加通顺。

尽管大多数情况下，你的答疑机器人是可以帮助发现问题的，但是现在的实现，仍存在一些问题：

1. **资源浪费**：对于检查文档错误的问题，大模型其实可以直接回复，并不需要检索参考资料，现在的实现存在资源浪费。
2. **误解问题**：因为现在每次会检索参考资料，这些被召回的相关文本段可能会干扰大模型理解问题，导致答非所问。

In [12]:
rag.ask('请帮我检查下这段文档：技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？', query_engine=query_engine)

是的，技术内容工程师需要设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等，确保内容符合教育标准和学习目标，同时考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。

### 4.1. 改进思路
为了解决上述问题，你可以想办法让这类问题直接输入给大模型，而不必经过 RAG 链路来生成答案。

同时，考虑到让同事只需要使用一个问答机器人，而无需在多个机器人之间切换，你可以借助大模型来识别问题的类型，然后根据不同的问题类型，在背后调用不同的大模型应用来生成答案。

<img src="https://img.alicdn.com/imgextra/i1/O1CN0126zKe71PAuJjWfQ3N_!!6000000001801-0-tps-2254-1080.jpg" width="800">

### 4.2. 实现一个简单的路由模块

你可以参考下面的代码，构建一个路由模块，用于识别问题类型。比如这里我们假设只有三种问题：

- 公司内部文档查询
- 内容翻译
- 文档审查

In [13]:
from chatbot import llm
prompt = '''
【角色背景】
你是一个问题分类路由器，需要识别问题的类型。
---
【任务要求】
问题的类型目前有：公司内部文档查询、内容翻译、文档审查
你只能选择一类意图作为你的判断，并不要说任何无关内容。
---
【用户输入】
以下是用户输入，请判断：
'''
def get_question_type(question):
    """
    获取问题的类型

    参数:
    - question (str): 需要判断的问题文本

    返回值:
    - str: 问题类型，可能是 "文档审查" 、 "内容翻译"或“公司内部文档查询”
    """
    return llm.invoke(prompt + question)

print(get_question_type('https://www.promptingguide.ai/zh/techniques/fewshot'))
print('\n')
print(get_question_type('That is a big one I dont know why'))
print('\n')
print(get_question_type('作为技术内容工程师有什么需要注意的吗？'))

内容翻译


无法明确分类，用户输入不清晰。 若要进行有效分类，请用户提供更具体的问题或请求。


公司内部文档查询


可以看到，虽然分类模块已经能够工作，但分类的输出准确度可能不太稳定。比如在上面的结果中，第二条分类出现了错误。

因此，接下来我们将尝试两种方法来优化分类结果：增加明确的输出格式、使用Few-shot技巧。通过这些优化，我们可以显著提升模型在分类任务中的表现，减少错误分类的发生。




### 4.3. 优化分类提示词
在上一节中，我们通过简单的提示词实现了问题的初步分类，但是分类的准确度不尽如人意。接下来，我们将进一步优化提示词，确保大模型能够更准确地对问题进行分类。

我们准备通过以下优化，提升分类的准确性：

- 明确输出格式：指定输出格式，使分类结果规范且易于解析。
- Few-shot 示例：提供示例，帮助大模型理解每个类别的特征和分类规则。

In [14]:
prompt = '''
【角色背景】
你是一个问题分类路由器，负责判断用户问题的类型，并将其归入下列三类之一：
1. 公司内部文档查询
2. 内容翻译
3. 文档审查

【任务要求】
你的任务是根据用户的输入内容，判断其意图并仅选择一个最贴切的分类。请仅输出分类名称，不需要多余的解释。判断依据如下：

- 如果问题涉及公司政策、流程、内部工具或职位描述与职责等内容，选择“公司内部文档查询”。
- 如果问题涉及任意一门非中文的语言，且输入中没有任何出现任何外语或出现“翻译”等字眼，选择“内容翻译”。
- 如果问题涉及检查或总结外部文档或链接内容，选择“文档审查”。
- 用户的前后输入与问题分类并没有任何关系，请单独为每次对话考虑分类类别。

【Few-shot 示例】
示例1：用户输入：“公司内部有哪些常用的项目管理工具？”
分类：公司内部文档查询

示例2：用户输入：“How can we finish the assignment on time?”
分类：内容翻译

示例3：用户输入：“https://help.aliyun.com/zh/model-studio/user-guide/long-context-qwen-long”
分类：文档审查

示例4：用户输入：“技术内容工程师需要设计和开发⾼质量的教育教材和课程吗？”
分类：文档审查

示例5：用户输入：“技术内容工程师核心职责是什么？”
分类：公司内部文档查询

【用户输入】
以下是用户的输入，请判断分类：

'''
print(get_question_type('https://www.promptingguide.ai/zh/techniques/fewshot'))
print('\n')
print(get_question_type('That is a big one I dont know why'))
print('\n')
print(get_question_type('作为技术内容工程师有什么需要注意的吗？'))

文档审查


内容翻译


公司内部文档查询


通过明确的输出格式和 few-shot 示例，机器人可以更准确地识别问题类型并输出符合预期的结果格式。这种优化让分类任务更加标准化，为接下来添加路由模块应用到问答机器人中打下了基础。

### 4.4. 将路由模块应用到问答机器人中

有了路由模块后，你就可以让问答机器人先识别问题的类型，在背后使用不同提示词和工作流程来回答问题。

In [17]:
def ask_v4(question):
    question_type = get_question_type(question)
    print(f'问题：{question}\n类型：{question_type}')
    if question_type == '文档审查':
        return llm.invoke('你是文档纠错专家，负责找出文档中或网页内容的明显错误，并且言简意赅的回复。如果没有明显问题，请直接回复没有问题\n' + question)
    elif question_type == '公司内部文档查询':
        return rag.ask(question, query_engine=query_engine)
    elif question_type == '内容翻译':
        return llm.invoke(f'你是一名翻译专家，你要识别不同语言的文本，并翻译为中文。\n{question}')
    else:
        return "未能识别问题类型，请重新输入。"

# 示例测试
query_engine = rag.create_query_engine(index=rag.load_index())
print(ask_v4('https://www.promptingguide.ai/zh/techniques/fewshot'))
print('')
print(ask_v4('请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'))
print('')
print(ask_v4('技术内容工程师有需要进行内容优化与更新与跨部门合作吗？'))
print('')
print(ask_v4('A true master always carries the heart of a student.'))

问题：https://www.promptingguide.ai/zh/techniques/fewshot
类型：文档审查
没有问题。

问题：请帮我检查下这段文档：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？
类型：文档审查
没有问题。

问题：技术内容工程师有需要进行内容优化与更新与跨部门合作吗？
类型：公司内部文档查询
技术内容工程师确实需要进行内容优化与更新，这包括根据学习者的反馈和评价来识别并调整内容中的潜在问题，以及定期更新材料以反映新的研究成果、技术进步和市场变化。此外，他们也需要与教学设计师、教育心理学家、技术团队及市场营销人员等多个部门紧密合作，确保内容的技术实施过程顺利进行，并有效传达给目标受众。这种跨部门的合作有助于共同创造出既具教育价值又具市场竞争力的产品。None

问题：A true master always carries the heart of a student.
类型：内容翻译
这是一句英文，翻译成中文是：“真正的高手总是怀有一颗学生的心。”


可以看到，有了一个路由模块后，你的答疑机器人可以更准确地回答问题了。

## ✅ 本节小结

通过学习本节课程，你已经掌握了提示词的基本框架和书写格式，并掌握了用大模型开发应用程序和算法应用的基本方法。此外，我们还学习了如何用大模型开发复杂的提示词应用。

实际落地大模型应用的过程中，领域专家的参与非常重要，甚至其中的提示词应该由领域专家来共同设计。因此，在你的工程代码中硬编码提示词，应该考虑调整成可配置的，甚至流程应该是可配置的，这样能更方便领域专家参与提示词设计和整个流程的设计。

阿里云百炼提供了可视化的大模型[应用构建](https://help.aliyun.com/zh/model-studio/user-guide/application-introduction#7c79befb2djg9)能力，可以在界面上完成提示词编写、甚至是整个复杂应用的流程可视化搭建，非常适合在需非技术背景的领域专家参与的大模型应用开发项目中。



## 🔥 课后小测验


【单选题】 2.2.1 以下哪个提示词要素用于明确要求大模型完成的任务？（ ）

A. 角色 (Role)

B. 受众 (Audience)

C. 任务目标 (Object)

D. 上下文 (Context)

答案：C

解释： 任务目标 (Object) 明确规定了大模型需要执行的操作或达成的结果。其他选项并非直接定义任务本身。角色 (Role) 定义大模型扮演的身份，受众 (Audience) 定义目标群体，上下文 (Context) 提供背景信息。

<br>

【多选题】 2.2.2. 假设你想使用大模型生成一段适合小学三年级学生阅读的关于太阳系的描述。以下哪个 user_query 的设计更合理？（ ）

A. user_query="Write about the solar system."

B. user_query="Tell me about the solar system."

C. user_query="Explain the solar system as if you were talking to a third-grade student."

D. user_query="Write a short and engaging description of the solar system for a third-grade audience, focusing on key planets and their characteristics."

E. user_query="太阳系是什么？"

答案: C, D (这两个选项明确指定了目标读者，并对内容和风格提出了要求。)

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。